In [1]:
# Source/Credit: The data for this project comes from a Kaggle dataset last updated 1 year ago by Devin Anzelmo.
# The dataset is available on Kaggle at: https://www.kaggle.com/devinanzelmo/dota-2-matches